## **[CNN 기반의 숫자 이미지 분류 - MNIST 데이터셋 활용_by Pytorch]**

## 1.라이브러리 가져오기

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import random

import warnings
warnings.filterwarnings("ignore")

## 2.데이터 로딩 및 전처리
- torchvision 라이브러리를 사용하여 MNIST 데이터셋을 다운로드
- train=True로 설정하면 훈련용 데이터셋(60,000개의 이미지)을 불러옵니다. False로 설정하면 테스트용 데이터셋(10,000개의 이미지)을 불러옵니다.
- transforms.ToTensor()는 이미지 데이터를 텐서로 변환합니다.
    - 이미지의 픽셀 값이 [0, 255] 범위에서 [0.0, 1.0] 범위로 정규화됨
    - MNIST 이미지는 원래 28x28 크기의 흑백 이미지로, 이를 [1, 28, 28] 크기의 3차원 텐서(채널, 높이, 너비)로 변환
- 배치 크기 단위로 데이터 로더를 사용해 데이터를 불러옵니다.

In [ ]:
# 배치 사이즈 설정
batch_size = 100
# 데이터 저장 경로 설정
root = './data'

# MNIST 학습/테스트 데이터셋 다운로드 및 로드, 텐서로 변환
mnist_train = dset.MNIST(root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST(root=root, train=False, transform=transforms.ToTensor(), download=True)


# 학습/테스 데이터 로더 설정 (데이터를 배치 크기만큼 나누고, 셔플하여 로드)
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, drop_last=True)

# GPU 사용 가능 시 CUDA를 사용하고, 그렇지 않으면 CPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device", device)

## 3.모델 정의하기

In [ ]:
# CNN 모델 정의
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # 첫 번째 Convolutional Layer: 입력 채널 1개, 출력 채널 32개, 커널 크기 3x3
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # 두 번째 Convolutional Layer: 입력 채널 32개, 출력 채널 64개, 커널 크기 3x3
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully Connected Layer: 7x7x64 -> 10개 클래스 출력
        self.fc = nn.Linear(7*7*64, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)  # Flatten the tensor
        out = self.fc(out)
        return out

# 모델 초기화
model = CNN().to(device)

# 모델 구조 출력
print(model)

- 모델 학습 과정을 위한 옵티마이저와 로스 함수 지정하기
    - 옵티마이저는 SGD, Loss는 Cross Entropy Loss를 사용합니다.

In [ ]:
# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

## 4.모델학습하기
- 구현 함수들을 이용해 학습 Loop를 구현해보세요.
    - 손실 함수로 CrossEntropyLoss를 사용하며, Adam 옵티마이저를 사용해 모델 파라미터를 업데이트합니다.

In [ ]:
# 학습 에포크 수 설정
training_epochs = 15

# 학습 과정의 손실과 정확도를 추적하기 위한 리스트 초기화
loss_history = []
accuracy_history = []

# 학습 루프 시작
for epoch in range(training_epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    total_batch = len(train_loader)

    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)

        # Forward pass
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 배치 손실 및 정확도 계산
        epoch_loss += loss.item()
        _, argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()
        epoch_accuracy += accuracy.item()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step[{}/{}, Loss: {:.4f}, Accuracy: {:.2f}%'.format(
                epoch + 1, training_epochs, i + 1, len(train_loader), loss.item(), accuracy.item() * 100
            ))

    epoch_loss /= total_batch
    epoch_accuracy /= total_batch
    loss_history.append(epoch_loss)
    accuracy_history.append(epoch_accuracy)

## 5.학습 과정 시각화

In [ ]:
# 학습 과정의 손실과 정확도를 그래프로 시각화
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(training_epochs), loss_history, label='Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')

plt.subplot(1, 2, 2)
plt.plot(range(training_epochs), accuracy_history, label='Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')

plt.show()

## 6.모델 평가하기
- 학습이 끝난 후 테스트 데이터셋을 사용해 모델의 성능을 평가하고, 임의로 선택한 이미지에 대한 예측 결과를 출력합니다.

In [ ]:
# 모델을 평가 모드로 전환
model.eval()

# 평가 시 그래디언트 계산하지 않음
with torch.no_grad():
    correct = 0
    total = 0

    for i, (t_imgs, t_labels) in enumerate(test_loader):
        t_imgs, t_labels = t_imgs.to(device), t_labels.to(device)

        prediction = model(t_imgs)
        _, argmax = torch.max(prediction, 1)
        total += t_imgs.size(0)
        correct += (t_labels == argmax).sum().item()

    print("Test accuracy for {} images: {:.2f}%".format(total, correct / total * 100))


## 7.모델 예측하기

In [ ]:
# 테스트 데이터의 임의 샘플에 대해 예측 결과 시각화
r = random.randint(0, len(mnist_test) - 1)
t_imgs_data = mnist_test.test_data[r: r + 1].float().to(device)
t_labels_data = mnist_test.test_labels[r:r + 1].to(device)

print("Label: ", t_labels_data.item())
single_prediction = model(t_imgs_data.unsqueeze(1))  # Add channel dimension
print("Prediction: ", torch.argmax(single_prediction, 1).item())

plt.imshow(mnist_test.test_data[r:r + 1].view(28, 28), cmap="Greys", interpolation="nearest")
plt.show()